In [1]:
import jieba, os, re
from gensim import corpora, models, similarities

def stopwordslist():
  stopwords = [line.strip() for line in open('./stopwords.txt', encoding='utf-8').readlines()]
  return stopwords

# stopwordslist()


def seg_depart(sentence):
  '''
  切分单个句子成分词, sentence输的句子
  '''
  jieba.load_userdict("../socialnet/name.txt")
  sentence_depart = jieba.cut(sentence.strip())
  stopwords = stopwordslist()
  outstr = ''
  
  for word in sentence_depart:
    if word not in stopwords:
      outstr += word
      outstr += " "
      
  return outstr


def generate_chapeters_from_book(file):
  '''
  从整本书中生成每一章
  '''
  chapter = ""
  lines = file.readlines()
  chapter_token = u"第[\u4E00-\u9FA5]+章"
  pattern = re.compile(chapter_token)
  num = 0 #章节数
  if not os.path.exists("./book"):
    os.makedirs("./book")
  for line in lines:
    if pattern.search(line) is None:
      chapter += line
    else:
      if num == 0:
        num += 1
        continue
      with open("./book/第{}章.txt".format(num),mode='w',encoding='utf-8') as f:
        f.write(chapter)
        
      chapter = ""
      num += 1
      
  with open("./book/第{}章.txt".format(num),mode='w',encoding='utf-8') as f:
    f.write(chapter)


with open("./book.txt",encoding='utf-8') as file:
  generate_chapeters_from_book(file)
    
    
def tokenize_chapter(chapter, chapter_name):
  '''
  对每个章节进行分词, 并保存到对应的文件中
  '''
  output_file = "分词_" + chapter_name
  if not os.path.exists("./分词"):
    os.makedirs("./分词")
  with open("./分词/"+output_file,mode='w',encoding='utf-8') as f:
    for line in chapter:
      line = re.sub(u'[^\u4e00-\u9fa5]+','',line)
      line_seg = seg_depart(line.strip())
      f.write(line_seg.strip()+'\n')
  
  print("=> 完成对{}内容的分词".format(chapter_name))

def tokenize():
  file_names = list(os.listdir("./book"))
  for file_name in file_names:
    with open("./book/"+file_name, mode='r', encoding='utf-8') as chapter:
      tokenize_chapter(chapter.readlines(), file_name)

  print("=> 分词完成")

tokenize()
  

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ljj12\Temp\jieba.cache
Loading model cost 2.194 seconds.
Prefix dict has been built successfully.


=> 完成对第100章.txt内容的分词
=> 完成对第101章.txt内容的分词
=> 完成对第102章.txt内容的分词
=> 完成对第103章.txt内容的分词
=> 完成对第104章.txt内容的分词
=> 完成对第105章.txt内容的分词
=> 完成对第106章.txt内容的分词
=> 完成对第107章.txt内容的分词
=> 完成对第108章.txt内容的分词
=> 完成对第109章.txt内容的分词
=> 完成对第10章.txt内容的分词
=> 完成对第110章.txt内容的分词
=> 完成对第111章.txt内容的分词
=> 完成对第112章.txt内容的分词
=> 完成对第113章.txt内容的分词
=> 完成对第114章.txt内容的分词
=> 完成对第115章.txt内容的分词
=> 完成对第116章.txt内容的分词
=> 完成对第117章.txt内容的分词
=> 完成对第118章.txt内容的分词
=> 完成对第119章.txt内容的分词
=> 完成对第11章.txt内容的分词
=> 完成对第120章.txt内容的分词
=> 完成对第121章.txt内容的分词
=> 完成对第122章.txt内容的分词
=> 完成对第123章.txt内容的分词
=> 完成对第124章.txt内容的分词
=> 完成对第125章.txt内容的分词
=> 完成对第126章.txt内容的分词
=> 完成对第127章.txt内容的分词
=> 完成对第128章.txt内容的分词
=> 完成对第129章.txt内容的分词
=> 完成对第12章.txt内容的分词
=> 完成对第130章.txt内容的分词
=> 完成对第131章.txt内容的分词
=> 完成对第132章.txt内容的分词
=> 完成对第133章.txt内容的分词
=> 完成对第134章.txt内容的分词
=> 完成对第135章.txt内容的分词
=> 完成对第136章.txt内容的分词
=> 完成对第137章.txt内容的分词
=> 完成对第138章.txt内容的分词
=> 完成对第139章.txt内容的分词
=> 完成对第13章.txt内容的分词
=> 完成对第140章.txt内容的分词
=> 完成对第141章.txt内容的分词
=> 完成对第142章.txt内容的分词
=> 完成对第143章.txt内容

In [74]:
dir = "./分词/"
tokenized_filenames = os.listdir(dir)

with open(dir+tokenized_filenames[0],mode='r',encoding='utf-8') as tokenized_chapter:
  tokenized_lines = []
  for line in tokenized_chapter:
    line = [word.strip() for word in line.split(' ')]
    tokenized_lines.append(line)
    
  # print(tokenized_line) 
  '''构建词频矩阵，训练LDA模型'''
  dictionary = corpora.Dictionary(tokenized_lines) 
  corpus = [dictionary.doc2bow(text) for text in tokenized_lines]
  
  lda = models.LdaModel(corpus=corpus,id2word=dictionary,num_topics=10)
  topic_list = lda.print_topics(10)
  print("10个主题的单词分布为：\n")
  for topic in topic_list:
    print(topic)


10个主题的单词分布为：

(0, '0.012*"李向前" + 0.012*"手术" + 0.008*"车" + 0.008*"黄原" + 0.008*"腿" + 0.008*"那位" + 0.008*"弟弟" + 0.008*"护士" + 0.008*"李登云" + 0.008*"想到"')
(1, '0.014*"一盘" + 0.014*"两" + 0.014*"老板" + 0.011*"说" + 0.011*"润生" + 0.009*"烧酒" + 0.008*"开车" + 0.008*"招呼" + 0.008*"耳朵" + 0.008*"凉拌"')
(2, '0.021*"值班" + 0.016*"司机" + 0.016*"门房" + 0.016*"急诊室" + 0.011*"向前" + 0.011*"外科" + 0.011*"护士" + 0.011*"下车" + 0.011*"老头" + 0.011*"跳"')
(3, '0.022*"医院" + 0.013*"地区" + 0.011*"这位" + 0.010*"司机" + 0.010*"面包车" + 0.009*"昏迷" + 0.008*"书记" + 0.008*"李登云" + 0.008*"卫生局" + 0.007*"李向前"')
(4, '0.019*"向前" + 0.013*"脸色" + 0.012*"汽车" + 0.009*"一半" + 0.009*"情况" + 0.008*"帮子" + 0.008*"喊" + 0.008*"惊叫" + 0.008*"难兄难弟" + 0.008*"停"')
(5, '0.019*"李向前" + 0.015*"汽车" + 0.012*"这位" + 0.012*"公司" + 0.011*"说" + 0.009*"运输" + 0.009*"原西县" + 0.009*"师傅" + 0.009*"里" + 0.008*"名字"')
(6, '0.017*"出车" + 0.013*"李向前" + 0.012*"里" + 0.011*"润生" + 0.010*"时间" + 0.010*"驾驶" + 0.008*"技术" + 0.008*"熟练" + 0.008*"单独" + 0.008*"让润生"')
(7, '0.015*"李登云" + 0.015*"一位" + 0.015*